In [1]:
from transformers import(
       AutoModelWithLMHead,
       AutoConfig,
       Trainer,
       AutoTokenizer,
       TextDataset,
       DataCollatorForLanguageModeling,
       TrainingArguments
)

In [ ]:
config = {
       "model_name":"gpt2",
       "train_dataset_filename":"combined.txt",
       "data_collator_block_size":256,
       "output_dir":".",
       "batch_size":8,
       "epochs":1
}

In [2]:

model = AutoModelWithLMHead.from_pretrained(config["model_name"])
tokenizer = AutoTokenizer.from_pretrained(config["model_name"])


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/transformers/models/auto/modeling_auto.py:966: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Downloading: 100%|██████████| 1.36M/1.36M [00:00<00:00, 3.10MB/s]


In [ ]:
dataset = TextDataset(tokenizer = tokenizer, file_path = config["train_dataset_filename"], block_size = config["data_collator_block_size"])
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [4]:
training_args = TrainingArguments(
       output_dir=config["output_dir"],
       per_device_train_batch_size=config["batch_size"],
       num_train_epochs=config["epochs"]
)

In [6]:
trainer = Trainer(
       model=model,
       args=training_args,
       data_collator=data_collator,
       train_dataset=dataset
)

W&B installed but not logged in. Run `wandb login` or set the WANDB_API_KEY env variable.


In [7]:
trainer.train()

  0%|          | 0/4286 [00:00<?, ?it/s]